In [1]:
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
import sys, importlib

In [2]:
# importlib.reload(sys.modules['pages.elements'])
# importlib.reload(sys.modules['scripts.preencher'])

from scripts.inicializar import Inicializar
from scripts.preencher import Preencher
# from pages.pages import PageFatura
# from pages.elements import *


In [3]:
browser = Firefox()

pagina = Inicializar(
    browser,
    'https://guardiaov4.seplag.ce.gov.br/auth',
    'credenciais.txt'
).run()

In [4]:
# pagina = PageFatura(browser)
p = Preencher(browser, 'data/SPS-10-2021.xlsx', intervalo_funcionarios=(11, 23), nome_planilha='Plan SPG')

In [5]:
p.dados.head()

,Nome,Dias Trabalhados,Salario Base,Salario Total,Adicional,Adicional Nortuno,Reserva,Encargos,Insalibridade,Periculosidade,...,Valor Hora Extra,DSR,Hora Encargos,Hora Taxa,Hora Tributos,Qtd Diarias,Passagem,Viagem,Viagem Taxa,Viagem Tributos
CPF,,,,,,,,,,,,,,,,,,,,,
667.797.073-68,Arthur Roberto Catunda Campos,30.0,2525.83,4966.32,0.0,0.0,0.0,1649.67,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
064.069.873-57,Deyvisson de Souza Bezerra,30.0,7332.89,13357.64,0.0,0.0,0.0,4789.26,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
460.441.831-49,Edson Garcia Nunes,30.0,2776.34,5405.54,0.0,0.0,0.0,1813.28,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
776.403.063-53,Fabiano Hugo Alves Maciel,30.0,9840.52,17791.23,0.0,0.0,0.0,6427.04,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
029.145.503-40,Felipe Augusto Moreira Frota,30.0,7332.89,13357.64,0.0,0.0,0.0,4789.26,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
p.run()

Log Positivo
Log Positivo
Log Positivo
Log Positivo
Log Positivo
Log Positivo
Log Positivo
Log Positivo
Log Positivo
